In [2]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2025',
                         user='postgres', 
                         password='1983')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS Expresso_2025 (
                                    DATA_CRIACAO      text,
                                    CENTRO    text,
                                    Giro text,
                                    LOPTICA  text,
                                    JANELA_HORARIA text,
                                    NOME text,
                                    MORADA text,
                                    CP text,
                                    LOCALIDADE text,
                                    COD_T_EVEN text,
                                    DATA_EVENTO text,
                                    LATITUDE text,
                                    LONGITUDE text,
                                    NOME_REM text,
                                    COD_PAIS_ORIGEM text,
                                    DATA_PA text) '''
criar_db(sql)


In [ ]:
# ==========================
# 1. Dump da base de dados PostgreSQL
# ==========================
import subprocess
import os

# Caminho do executável pg_dump
PG_DUMP_PATH = r"C:\Program Files\PostgreSQL\17\bin\pg_dump.exe"  # ajusta se for outra versão

# Caminho do ficheiro de saída
output_file = os.path.expanduser(r"~\Downloads\backup_ctt2025_09.dump")

# Comando para executar o dump
cmd = [
    PG_DUMP_PATH,
    "-h", "localhost",
    "-U", "postgres",
    "-p", "5432",
    "-d", "ctt2025",
    "-F", "c",  # formato custom (para pg_restore)
    "-f", output_file
]

# Executa o comando no Windows pedindo a password
subprocess.run(cmd, env={**os.environ, "PGPASSWORD": "1983"})


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [2]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor()
cur.execute("SELECT DISTINCT CONCAT(SUBSTRING(DATA_CRIACAO, 7, 4),SUBSTRING(DATA_CRIACAO, 4, 2),SUBSTRING(DATA_CRIACAO, 1, 2)) AS ConvertedDate FROM Expresso_2025")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20250621' '20250622' '20250623' '20250624' '20250625' '20250626'
 '20250627' '20250628' '20250629' '20250630' '20250701' '20250702'
 '20250703' '20250704' '20250705' '20250706' '20250707' '20250708'
 '20250709' '20250731' '20250801' '20250802' '20250803' '20250804'
 '20250805' '20250806' '20250807' '20250808' '20250809' '20250810'
 '20250811' '20250812' '20250813' '20250814' '20250815' '20250816'
 '20250817' '20250818' '20250819' '20250820' '20250821' '20250822'
 '20250823' '20250824' '20250825' '20250826' '20250827' '20250828'
 '20250829' '20250830' '20250831' '20250901' '20250902' '20250903'
 '20250904' '20250905' '20250906' '20250907' '20250908' '20250909'
 '20250910' '20250911' '20250912' '20250913' '20250914' '20250915'
 '20250916' '20250917' '20250918' '20250919' '20250920' '20250921'
 '20250922' '20250923' '20250924' '20250925' '20250926' '20250927'
 '20250928' '20250929' '20250930' '20251001' '20251002' '20251003'
 '20251004' '20251005' '20251006' '20251007']


Expresso

In [3]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/EventosPDA')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM Expresso_2025 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        ano = file[14:18]
        mes = date_str_s[4:6]
        date_str_s_array = np.array(date_str_s)
        
        if date_str_s not in data_validation and ano == "2025" and mes >= "05":
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';', on_bad_lines='skip', encoding="Latin-1")
            ftp.quit()
            
            if len(data_rows) == 1: 
                continue
            else:
                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO Expresso_2025 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1}: {error}")
                        con.rollback()
                    else:
                        con.commit()
                        
                end_time = time.time()    
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")

con.close()


File 'EVPDA3466_ENV_20251008_20251009010517.CSV' inserted into the database: 806.9606306552887 seconds
File 'EVPDA3466_ENV_20251009_20251010010500.CSV' inserted into the database: 709.0417113304138 seconds


In [4]:
'''
from ftplib import FTP
import io
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

# --- ligações pré-existentes ---
# con = psycopg2.connect(...)

# 1) validar dias já carregados (usa set para O(1))
with con.cursor() as cur:
    cur.execute("""
        SELECT DISTINCT SUBSTRING(DATA_CRIACAO, 7, 4) || SUBSTRING(DATA_CRIACAO, 4, 2) || SUBSTRING(DATA_CRIACAO, 1, 2) AS dias
        FROM Expresso_2025
    """)
    data_validation = {r[0] for r in cur.fetchall()}

# 2) abrir uma única sessão FTP
ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')
ftp.cwd('/OP/Rotas/EventosPDA')
files = ftp.nlst()

def process_data(row):
    try:
        return row.decode('Latin-1').strip()
    except UnicodeDecodeError:
        return row.decode('utf-8').strip()

def copy_df(conn, df, table):
    # COPY CSV para Postgres (muito mais rápido que INSERT por linha)
    buf = io.StringIO()
    # Se o teu CSV no destino deve usar ';', mantém sep=';'
    df.to_csv(buf, index=False, header=False, sep=';')
    buf.seek(0)
    with conn.cursor() as c:
        c.copy_expert(f"COPY {table} ({', '.join(df.columns)}) FROM STDIN WITH (FORMAT csv, DELIMITER ';')", buf)

# 3) loop de ficheiros
for file in files:
    if not file.startswith("E"):
        continue

    date_str_s = file[14:22].replace("/", "")
    ano = file[14:18]
    mes = date_str_s[4:6]

    # filtros rápidos para evitar trabalho desnecessário
    if not (ano == "2025" and mes >= "05"):
        continue
    if date_str_s in data_validation:
        continue

    start_time = time.time()

    # ler ficheiro do FTP uma única vez
    file_data = io.BytesIO()
    ftp.retrbinary('RETR ' + file, file_data.write)
    file_data.seek(0)

    # processar linhas (ignorando a última se for trailer)
    lines = file_data.readlines()
    if len(lines) <= 1:
        continue
    data_rows = [process_data(row) for row in lines[:-1]]

    # construir DataFrame (um parse por ficheiro)
    df = pd.read_csv(io.StringIO('\n'.join(data_rows)), sep=';', on_bad_lines='skip', encoding="Latin-1")

    # normalizar NaN -> vazio (COPY trata bem) ou, se precisares de NULL, substitui por ''
    df = df.where(pd.notna(df), None)

    # 4) transação por ficheiro + COPY
    try:
        con.autocommit = False
        copy_df(con, df, "Expresso_2025")
        con.commit()
        data_validation.add(date_str_s)  # marca como carregado
        print(f"File '{file}' -> {len(df)} linhas, {time.time() - start_time:.1f}s")
    except Exception as e:
        con.rollback()
        print(f"ERRO em '{file}': {e}")

# fechar FTP e DB
ftp.quit()
con.close()
'''

'\nfrom ftplib import FTP\nimport io\nimport pandas as pd\nimport numpy as np\nimport psycopg2\nimport psycopg2.extras\nimport time\n\n# --- ligações pré-existentes ---\n# con = psycopg2.connect(...)\n\n# 1) validar dias já carregados (usa set para O(1))\nwith con.cursor() as cur:\n    cur.execute("""\n        SELECT DISTINCT SUBSTRING(DATA_CRIACAO, 7, 4) || SUBSTRING(DATA_CRIACAO, 4, 2) || SUBSTRING(DATA_CRIACAO, 1, 2) AS dias\n        FROM Expresso_2025\n    """)\n    data_validation = {r[0] for r in cur.fetchall()}\n\n# 2) abrir uma única sessão FTP\nftp = FTP()\nftp.connect(host=\'10.0.25.193\', port=2122)\nftp.login(user=\'cax_sgc\', passwd=\'gy768#lo\')\nftp.cwd(\'/OP/Rotas/EventosPDA\')\nfiles = ftp.nlst()\n\ndef process_data(row):\n    try:\n        return row.decode(\'Latin-1\').strip()\n    except UnicodeDecodeError:\n        return row.decode(\'utf-8\').strip()\n\ndef copy_df(conn, df, table):\n    # COPY CSV para Postgres (muito mais rápido que INSERT por linha)\n    buf = 

BANCA 

In [5]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2025',
                         user='postgres', 
                         password='1983')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS BANCA_2025 (
                                    COD_CLIENTE_BNC text,
                                    CODIGO_BALCAO_SIDIR text,
                                    CODIGO_BALCAO_AF text,
                                    AGENCIA text,
                                    CA text,
                                    CIRCUITO text,
                                    DATA text,
                                    CONTACTO text,
                                    FATURAR text,
                                    ENTREGUE text,
                                    TAREFA text,
                                    CP text,
                                    CENTRO_OPERACIONAL text,
                                    MORADA_TOQUE text,
                                    HORA_TOQUE_PREVISTA text,
                                    HORA_TOQUE_REAL text
                                    ) '''
criar_db(sql)





In [6]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct REPLACE(data, '.', '') dias FROM Banca_2025 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20250624' '20250625' '20250626' '20250627' '20250630' '20250701'
 '20250702' '20250703' '20250704' '20250707' '20250708' '20250709'
 '20250710' '20250711' '20250714' '20250715' '20250716' '20250717'
 '20250718' '20250721' '20250722' '20250723' '20250724' '20250725'
 '20250728' '20250729' '20250730' '20250731' '20250801' '20250804'
 '20250805' '20250806' '20250807' '20250808' '20250811' '20250812'
 '20250813' '20250814' '20250815' '20250818' '20250819' '20250820'
 '20250821' '20250822' '20250825' '20250826' '20250827' '20250828'
 '20250829' '20250901' '20250902' '20250903' '20250904' '20250905'
 '20250908' '20250909' '20250910' '20250911' '20250912' '20250915'
 '20250916' '20250917' '20250918' '20250919' '20250922' '20250923'
 '20250924' '20250925' '20250926' '20250929' '20250930' '20251001'
 '20251002' '20251003']


In [7]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Banca_Recolhas')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

con = psycopg2.connect(host='localhost', 
                    database='ctt2025',
                    user='postgres', 
                    password='1983')

cur = con.cursor()

cur.execute("SELECT distinct REPLACE(data, '.', '') dias FROM Banca_2025 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

# Create a mapping between input column names and database column names
column_mapping = {
    'ï»¿COD_CLIENTE_BNC': 'COD_CLIENTE_BNC',
    'CODIGO_BALCAO_SIDIR': 'CODIGO_BALCAO_SIDIR',
    'CODIGO_BALCAO_AF': 'CODIGO_BALCAO_AF',
    'AGENCIA': 'AGENCIA',
    'CA': 'CA',
    'CIRCUITO': 'CIRCUITO',
    'DATA': 'DATA',
    'CONTACTO': 'CONTACTO',
    'FATURAR': 'FATURAR',
    'ENTREGUE': 'ENTREGUE',
    'TAREFA': 'TAREFA',
    'CP': 'CP',
    'CENTRO OPERACIONAL': 'CENTRO_OPERACIONAL',
    'MORADA TOQUE': 'MORADA_TOQUE',
    'HORA TOQUE PREVISTA': 'HORA_TOQUE_PREVISTA',
    'HORA TOQUE REAL': 'HORA_TOQUE_REAL'
}

for file in files:
    start_time = time.time()
    if file.startswith("C"):
        date_str_s = file[21:29].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        ano = file[21:25]
        if date_str_s not in data_validation and ano =="2025":
            
            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')
            
            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line
            
            # Remove BOM if present
            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]
            
            # Join the data_rows without the last line (to exclude the empty line) and convert to bytes
            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')
            
            # Decode the bytes data using utf-8-sig to handle the BOM
            decoded_data = csv_data.decode('utf-8-sig')
            
            if decoded_data.strip():  # Check if the decoded data is not empty
                # Create a DataFrame from the cleaned data rows
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', on_bad_lines='skip', encoding="Latin-1")

                # Rename the DataFrame columns based on the mapping
                df.rename(columns=column_mapping, inplace=True)

                # Insert the data into the database
                try:
                    cur = con.cursor()
                    # Replace "NaN" values with None (NULL) for database insertion
                    df = df.where(pd.notna(df), None)

                    # Generate placeholders for the query based on the number of columns
                    placeholders = ', '.join(['%s'] * len(df.columns))

                    # Construct the INSERT query with placeholders
                    insert_query = f"INSERT INTO Banca_2025 ({', '.join(df.columns)}) VALUES ({placeholders})"

                    # Execute the query with the DataFrame values as parameters
                    psycopg2.extras.execute_batch(cur, insert_query, df.values)
                    con.commit()

                    end_time = time.time()
                    print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")
                except (Exception, psycopg2.DatabaseError) as error:
                    print(f"Error inserting data from file '{file}': {error}")
                    con.rollback()

ftp.quit()
con.close()


File 'CAX019_TOQUE_BANCA_D_20251006_20251007000639.CSV' inserted into the database: 19.024969577789307 seconds
File 'CAX019_TOQUE_BANCA_D_20251007_20251008000608.CSV' inserted into the database: 0.7659471035003662 seconds
File 'CAX019_TOQUE_BANCA_D_20251008_20251009000635.CSV' inserted into the database: 1.2202017307281494 seconds
File 'CAX019_TOQUE_BANCA_D_20251009_20251010000624.CSV' inserted into the database: 0.727358341217041 seconds


Recolhas

In [8]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2025',
                         user='postgres', 
                         password='1983')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = '''CREATE TABLE IF NOT EXISTS Recolhas_2025 (
            "data_ficheiro" text,
            "Código Orgânico" text,
            "Descrição" text,
            "Dia" text,
            "Giro" text,
            "Tipo" text,
            "Ponto Recolha" text,
            "Nome Cliente" text,
            "Morada de recolha" text,
            "Código postal recolha" text,
            "Qntd de Objetos Recolhidos" text,
            "Estado" text,
            "Hora Início" text,
            "Hora Fim" text,
            "Hora Real" text
        )'''


criar_db(sql)



In [9]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)

cur.execute("SELECT DISTINCT  data_ficheiro as dias FROM Recolhas_2025  ORDER BY DIAS ASC")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20250624' '20250625' '20250626' '20250627' '20250628' '20250629'
 '20250630' '20250701' '20250702' '20250703' '20250704' '20250705'
 '20250706' '20250707' '20250708' '20250709' '20250710' '20250711'
 '20250712' '20250713' '20250714' '20250715' '20250716' '20250717'
 '20250718' '20250719' '20250720' '20250721' '20250722' '20250723'
 '20250724' '20250725' '20250726' '20250727' '20250728' '20250729'
 '20250730' '20250731' '20250801' '20250802' '20250803' '20250804'
 '20250805' '20250806' '20250807' '20250808' '20250809' '20250810'
 '20250811' '20250812' '20250813' '20250814' '20250815' '20250816'
 '20250817' '20250818' '20250819' '20250820' '20250821' '20250822'
 '20250823' '20250824' '20250825' '20250826' '20250827' '20250828'
 '20250829' '20250830' '20250831' '20250901' '20250902' '20250903'
 '20250904' '20250905' '20250906' '20250907' '20250908' '20250909'
 '20250910' '20250911' '20250912' '20250913' '20250914' '20250915'
 '20250916' '20250917' '20250918' '20250919' '20250920' '20250

In [10]:
import pandas as pd
import numpy as np
import io
import time
from ftplib import FTP
import psycopg2
import psycopg2.extras
import datetime


current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/Banca_Recolhas')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')
    return decoded_line.strip()

data_validation = []

con = psycopg2.connect(host='localhost', 
                    database='ctt2025',
                    user='postgres', 
                    password='1983')

cur = con.cursor()

cur.execute("SELECT DISTINCT  data_ficheiro as dias FROM Recolhas_2025  ORDER BY DIAS ASC")

for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

# List of column names
columns = ['"Código Orgânico"', '"Descrição"', '"Dia"', '"Giro"', '"Tipo"', '"Ponto Recolha"', '"Nome Cliente"',
           '"Morada de recolha"', '"Código postal recolha"', '"Qntd de Objetos Recolhidos"', '"Estado"', '"Hora Início"',
           '"Hora Fim"', '"Hora Real"']

# Assuming you have a 'files' list defined earlier
for file in files:
    start_time = time.time()
    if file.startswith("R"):
        date_str_s = file[19:27].replace("/", "")
        date_str_s_array = np.array(date_str_s)
        ano = file[19:23]
        if date_str_s not in data_validation and ano == "2025":

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/Banca_Recolhas')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            # Process the data line by line, excluding the last line
            lines = file_data.readlines()
            data_rows = [process_data(row) for row in lines[:-1]]  # Exclude the last line

            # Remove BOM if present
            if data_rows and data_rows[0].startswith('\ufeff'):
                data_rows[0] = data_rows[0][1:]

            # Join the data_rows without the last line (to exclude the empty line) and convert to bytes
            csv_data = '\n'.join(data_rows[:-1]).encode('utf-8')

            # Decode the bytes data using utf-8-sig to handle the BOM
            decoded_data = csv_data.decode('utf-8-sig')

            if decoded_data.strip():  # Check if the decoded data is not empty
                # Create a DataFrame from the cleaned data rows, specifying column names
                df = pd.read_csv(io.StringIO(decoded_data), sep=';', names=columns, on_bad_lines='skip', encoding="Latin-1")
                df = df.iloc[1:]

                # Add the 'data_ficheiro' column and fill it with date_str_s value
                df['data_ficheiro'] = date_str_s

                # Insert the data into the database
                try:
                    # Replace "NaN" values with None (NULL) for database insertion
                    df = df.where(pd.notna(df), None)

                    # Generate placeholders for the query based on the number of columns
                    placeholders = ', '.join(['%s'] * len(df.columns))

                    # Construct the INSERT query with placeholders
                    insert_query = f"INSERT INTO Recolhas_2025 ({', '.join(df.columns)}) VALUES ({placeholders})"

                    # Execute the query with the DataFrame values as parameters
                    psycopg2.extras.execute_batch(cur, insert_query, df.values)
                    con.commit()

                    end_time = time.time()
                    print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")
                except (Exception, psycopg2.DatabaseError) as error:
                    print(f"Error inserting data from file '{file}': {error}")
                    con.rollback()


ftp.quit()
con.close()


File 'RECSG4366_RECOLHAS_20251006_20251007010501.CSV' inserted into the database: 6.681854248046875 seconds
File 'RECSG4366_RECOLHAS_20251007_20251008010501.CSV' inserted into the database: 27.404104471206665 seconds
File 'RECSG4366_RECOLHAS_20251008_20251009010500.CSV' inserted into the database: 18.166507244110107 seconds
File 'RECSG4366_RECOLHAS_20251009_20251010010500.CSV' inserted into the database: 2.8839011192321777 seconds


REDE BASE


In [11]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='ctt2025',
                         user='postgres', 
                         password='1983')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

sql = ''' CREATE TABLE IF NOT EXISTS REDE_BASE_2025 (
                                    DATA_CRIACAO      text,
                                    CENTRO    text,
                                    Giro text,
                                    LOPTICA  text,
                                    JANELA_HORARIA text,
                                    NOME text,
                                    MORADA text,
                                    CP text,
                                    LOCALIDADE text,
                                    COD_T_EVEN text,
                                    DATA_EVENTO text,
                                    LATITUDE text,
                                    LONGITUDE text,
                                    NOME_REM text,
                                    COD_PAIS_ORIGEM text) '''
criar_db(sql)


In [12]:
import numpy as np
data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM REDE_BASE_2025 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)
print(data_validation)

['20250624' '20250625' '20250626' '20250627' '20250628' '20250630'
 '20250701' '20250702' '20250703' '20250704' '20250705' '20250707'
 '20250708' '20250709' '20250710' '20250711' '20250712' '20250713'
 '20250714' '20250715' '20250716' '20250717' '20250718' '20250719'
 '20250720' '20250721' '20250722' '20250723' '20250724' '20250725'
 '20250726' '20250727' '20250728' '20250729' '20250730' '20250731'
 '20250801' '20250802' '20250803' '20250804' '20250805' '20250806'
 '20250807' '20250808' '20250809' '20250810' '20250811' '20250812'
 '20250813' '20250814' '20250815' '20250816' '20250817' '20250818'
 '20250819' '20250820' '20250821' '20250822' '20250823' '20250824'
 '20250825' '20250826' '20250827' '20250828' '20250829' '20250830'
 '20250831' '20250901' '20250902' '20250903' '20250904' '20250905'
 '20250906' '20250908' '20250909' '20250910' '20250911' '20250912'
 '20250913' '20250914' '20250915' '20250916' '20250917' '20250918'
 '20250919' '20250920' '20250921' '20250922' '20250923' '20250

In [13]:
from ftplib import FTP
import os
import io
import datetime
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import time

current_year = datetime.datetime.now().year

ftp = FTP()
ftp.connect(host='10.0.25.193', port=2122)
ftp.login(user='cax_sgc', passwd='gy768#lo')

ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')

files = ftp.nlst()

def process_data(row):
    try:
        decoded_line = row.decode('Latin-1')
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError in file '{file}', line: {row}")
        decoded_line = row.decode('utf-8')

    # Check if the number of columns is 13 (assuming semicolon as the delimiter)
    if len(decoded_line.split(';')) == 13:
        return decoded_line.strip()
    else:
        print(f"Ignoring row with incorrect number of columns: {decoded_line}")
        return None  # Skip the row if it has an incorrect number of columns

data_validation = []

cur = con.cursor()

cur.execute("SELECT distinct  SUBSTRING(DATA_CRIACAO, 7, 4) ||  SUBSTRING(DATA_CRIACAO, 4, 2) ||  SUBSTRING(DATA_CRIACAO, 1, 2) as dias FROM REDE_BASE_2025 order by dias asc")
for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])

cur.close()

for file in files:
    start_time = time.time()
    if file.startswith("E"):
        date_str_s = file[14:22].replace("/", "")
        month = file[18:20].replace("/", "")
        date_str_s = np.array(date_str_s)
        ano = file[14:18]
        if date_str_s not in data_validation and month >= "01" and ano == "2025":
            print(date_str_s)

            ftp = FTP()
            ftp.connect(host='10.0.25.193', port=2122)
            ftp.login(user='cax_sgc', passwd='gy768#lo')
            ftp.cwd('/OP/Rotas/EventosPDA_Rede_base')

            file_data = io.BytesIO()
            ftp.retrbinary('RETR ' + file, file_data.write)
            file_data.seek(0)  # Reset the stream position to the beginning

            try:
                df = pd.read_csv(file_data, sep=';', on_bad_lines='skip', encoding="Latin-1")
            except pd.errors.ParserError as pe:
                print(f"ParserError occurred while reading CSV in file '{file}':")
                print(pe)
                print(f"Skipping file '{file}'.")
                continue

            ftp.quit()

            if len(df) == 0:
                print(f"All rows in file '{file}' were problematic. Skipping insertion.")
                continue
            else:
                # Create a DataFrame from the cleaned data rows

                # Insert the data into the database
                for index, row in df.iterrows():
                    try:
                        cur = con.cursor()
                        # Replace "NaN" values with None (NULL) for database insertion
                        row = [None if pd.isna(value) else value for value in row]
                        placeholders = ', '.join(['%s'] * len(row))
                        insert_query = f"INSERT INTO REDE_BASE_2025 ({', '.join(df.columns)}) VALUES ({placeholders})"
                        cur.execute(insert_query, tuple(row))
                    except (Exception, psycopg2.DatabaseError) as error:
                        print(f"Error inserting row {index+1} from file '{file}': {error}")
                        con.rollback()
                    else:
                        con.commit()

                end_time = time.time()
                print(f"File '{file}' inserted into the database: {end_time - start_time} seconds")

con.close()


20250928
File 'EVRBP4348_ENV_20250928_20250929010503.CSV' inserted into the database: 0.4058809280395508 seconds
20251005
File 'EVRBP4348_ENV_20251005_20251006010513.CSV' inserted into the database: 0.6961860656738281 seconds
20251006
File 'EVRBP4348_ENV_20251006_20251007010501.CSV' inserted into the database: 519.7565469741821 seconds
20251007
File 'EVRBP4348_ENV_20251007_20251008010502.CSV' inserted into the database: 572.1092419624329 seconds
20251008
File 'EVRBP4348_ENV_20251008_20251009010517.CSV' inserted into the database: 696.5280070304871 seconds
20251009
File 'EVRBP4348_ENV_20251009_20251010010500.CSV' inserted into the database: 792.761011838913 seconds
